# ## second clustering approach: pair of keywords 
since that I use a new app dataset (that is the union of the Official Samsung SmartThings: https://github.com/vomar18/SmartThingsPublic and the nsslabcuus: https://github.com/nsslabcuus/IoTMon)
1. check every single app description in order to identify:"The entity keyword with the highest aggregated value is considered as a representative keyword for the cluster"
2. create the distance and then the similarity matrix
3. save the pair of word with the highest aggregation value (over than 90%??)
4. save the result inside: "analysis_NEW/physical_channel_2.JSON"

In [1]:
# !!attention!! use python 3.6 for manage the packet stanza that is 
# the Official Stanford NLP Python Library for Many Human Languages

from gensim.models import Word2Vec, KeyedVectors 
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist # for the Manhattan distance
import json   # for read the JSON file
import stanza # Official Stanford NLP Python Library for Many Human Languages
import re     # for preprocessing the descriptions (necessary for using the word2vec)
import numpy as np

In [2]:
#read only descriptions and save them
descriptions = [] #array with descriptions
with open("", "r") as read_file: # load the JSON file with app descriptions
    app_descriptions = json.load(read_file)

for i in app_descriptions:
    descriptions.append(app_descriptions[i])    

In [3]:
print("total app description found: "+ str(len(descriptions)))

total app description found: 129


In [4]:
stanza.download('en')       # This downloads the English models for the neural pipeline
nlp = stanza.Pipeline('en') # This sets up a default neural pipeline in English
model = KeyedVectors.load_word2vec_format('/home/volta/Documenti/CCS18_CS/word2vec/GoogleNews-vectors-negative300.bin', binary=True, limit=100000) # load word2vec pre-trained

2021-09-17 08:18:44 INFO: Downloading default packages for language: en (English)...
2021-09-17 08:18:46 INFO: File exists: /home/volta/stanza_resources/en/default.zip.
2021-09-17 08:18:51 INFO: Finished downloading models and saved to /home/volta/stanza_resources.
2021-09-17 08:18:51 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| pos       | combined  |
| lemma     | combined  |
| depparse  | combined  |
| sentiment | sstplus   |
| ner       | ontonotes |

2021-09-17 08:18:51 INFO: Use device: cpu
2021-09-17 08:18:51 INFO: Loading: tokenize
2021-09-17 08:18:51 INFO: Loading: pos
2021-09-17 08:18:52 INFO: Loading: lemma
2021-09-17 08:18:52 INFO: Loading: depparse
2021-09-17 08:18:52 INFO: Loading: sentiment
2021-09-17 08:18:53 INFO: Loading: ner
2021-09-17 08:18:53 INFO: Done loading processors!


In [7]:
set_of_keywords = []
POS_accepted=["NN"] # part of speech tags accepted ---> deleted: "NNP", "NNPS"

for i in range(len(descriptions)):
    keywords = []       # keywords found
    keyword_vector = {} # dictionary keyword -> vector rappresentation
    print("analysing app description: ",i)
    des_analysed = nlp(descriptions[i]) # apply Stanford NLP to description
    for sent in des_analysed.sentences:
        for word in sent.words: # check each word of the description
            if word.xpos in POS_accepted:
                keywords_found = word.text.lower() # filter: to all lowercase words
                if not keywords_found in keywords: # if not already save that word
                    print(f"word:{word.text}")
                    keywords.append(keywords_found) # add to the list of keywords
    if len(keywords) > 0:
        for i in range(len(keywords)):
            try:
                word = str(keywords[i])
                if len(word) > 1:
                    keyword_vector[word] = model[word] # using Word2Vec
                else:
                    print("word not processed: ",word)
            except KeyError as err:
                print("handling Error: ", err) # some words can not be processed by Word2Vec

        similarity_matrix = np.ones((len(keyword_vector.keys()), len(keyword_vector.keys())))
        similarity_matrix = similarity_matrix * 999
        keyword_lsit = list(keyword_vector.keys())
        print(keyword_lsit)
        #print(similarity_matrix)
        for key_x in range(len(keyword_vector.keys())):
            for key_y in range(len(keyword_vector.keys())):
                #print("check",keywords[key_x], " and ", keywords[key_y] )
                if key_x != key_y:
                    similarity_matrix[key_x][key_y] = (np.linalg.norm(keyword_vector[keyword_lsit[key_x]]-keyword_vector[keyword_lsit[key_y]]))
        
        #print(similarity_matrix)
        aggregation_result = np.min(similarity_matrix, axis=0)
        key1 = keyword_lsit[np.argmin(aggregation_result)]
        key2 = keyword_lsit[np.argmin(similarity_matrix[np.argmin(aggregation_result)][:])]
        if not key1 in set_of_keywords:
            set_of_keywords.append(key1)
        if not key2 in set_of_keywords:
            set_of_keywords.append(key2)
        
#         keywords = list(keyword_vector.keys())
#         vectors = []
#         for key in keyword_vector.keys():
#             vectors.append(keyword_vector[key])

#         centroid = np.array(vectors).mean(axis=0) # axis=0 all vector 300x1
#         #print(centroid)
#         similarity = []
#         for key in keyword_vector.keys():
#             similarity.append(np.linalg.norm(centroid-keyword_vector[key]))
#         print("win:",keywords[np.argmin(similarity)])
#         if not keywords[np.argmin(similarity)] in set_of_keywords:
#             set_of_keywords.append(keywords[np.argmin(similarity)])
            
#set_of_keywords


#aggregation_result = np.sum(similarity_matrix, axis=0)
#set_of_keywords.append(keywords[np.argmin(aggregation_result)])
    
#set_of_keywords

analysing app description:  0
word:graph
['graph']
analysing app description:  1
word:thermostat
word:everything
['thermostat', 'everything']
analysing app description:  2
word:Home
word:phrase
word:door
word:region
['home', 'phrase', 'door', 'region']
analysing app description:  3
word:humidity
word:threshold
word:switch
['humidity', 'threshold', 'switch']
analysing app description:  4
analysing app description:  5
analysing app description:  6
word:set
word:Presence
word:mode
word:change
word:everyone
word:conjunction
word:Turn
word:thermostat
word:security
['set', 'presence', 'mode', 'change', 'everyone', 'conjunction', 'turn', 'thermostat', 'security']
analysing app description:  7
word:motion
['motion']
analysing app description:  8
word:place
word:home
['place', 'home']
analysing app description:  9
word:sensor
word:space
['sensor', 'space']
analysing app description:  10
word:Control
['control']
analysing app description:  11
word:power
word:time
['power', 'time']
analysing app 

word:push
word:notification
word:Presence
word:tag
word:smartphone
word:location
['push', 'notification', 'presence', 'tag', 'smartphone', 'location']
analysing app description:  74
word:text
word:message
word:presence
word:status
['text', 'message', 'presence', 'status']
analysing app description:  75
word:inclement
word:weather
handling Error:  "Key 'inclement' not present"
['weather']
analysing app description:  76
word:arrival
word:departure
word:car
word:door
word:N
word:motion
word:interior
word not processed:  n
['arrival', 'departure', 'car', 'door', 'motion', 'interior']
analysing app description:  77
word:someone
word:time
word:morning
['someone', 'time', 'morning']
analysing app description:  78
word:time
word:day
['time', 'day']
analysing app description:  79
word:push
word:notification
word:weather
word:area
['push', 'notification', 'weather', 'area']
analysing app description:  80
word:mode
word:candle
word:lighting
word:havdalah
word:API
word:shabbat
word:chag
word:time


In [8]:
set_of_keywords.sort()
set_of_keywords

['account',
 'air',
 'alarm',
 'amount',
 'anything',
 'anytime',
 'app',
 'area',
 'arrival',
 'bed',
 'burst',
 'button',
 'case',
 'change',
 'chat',
 'collection',
 'color',
 'contact',
 'control',
 'controller',
 'cube',
 'day',
 'demonstration',
 'departure',
 'device',
 'door',
 'element',
 'end',
 'endpoint',
 'energy',
 'event',
 'everything',
 'garage',
 'graph',
 'home',
 'hour',
 'house',
 'hub',
 'hue',
 'humidity',
 'input',
 'jawbone',
 'level',
 'life',
 'light',
 'lighting',
 'location',
 'mail',
 'mailbox',
 'manager',
 'message',
 'middle',
 'mod',
 'mode',
 'money',
 'mood',
 'motion',
 'movement',
 'multi',
 'night',
 'notification',
 'number',
 'option',
 'panel',
 'panic',
 'period',
 'person',
 'phone',
 'place',
 'power',
 'presence',
 'push',
 'region',
 'report',
 'row',
 'schedule',
 'sensor',
 'service',
 'set',
 'setting',
 'setup',
 'shed',
 'sleep',
 'someone',
 'something',
 'song',
 'sound',
 'space',
 'speaker',
 'state',
 'station',
 'status',
 'step

In [9]:
np_keywords = []
for i in range(len(set_of_keywords)):
    np_keywords.append(model[set_of_keywords[i]])
np_keywords = np.array(np_keywords)

In [10]:
print(f"tot keywords:{np_keywords.shape[0]}, tot features:{np_keywords.shape[1]}") # check the numpy array

tot keywords:112, tot features:300


In [11]:
# start the clustering algh
n_cluster = 16 # by paper
np.random.seed(1)
initial_cluster_idx = np.random.choice(len(np_keywords), n_cluster, replace=False)
initial_cluster_idx # initial random centroids

array([ 91,  44,  56,  82,  69, 103,  99,  58,  35,  38,  65,  92,   2,
       100, 105,  53])

In [12]:
centroid_coordinate = np_keywords[initial_cluster_idx,:]
centroid_coordinate # coordinate of initial ranfom centroids

array([[ 0.09130859,  0.08837891,  0.03881836, ...,  0.08447266,
         0.09130859, -0.07421875],
       [ 0.12988281,  0.17382812,  0.10302734, ...,  0.10009766,
         0.19726562, -0.11865234],
       [ 0.06884766,  0.05078125, -0.19140625, ..., -0.05395508,
        -0.09521484,  0.2265625 ],
       ...,
       [ 0.0456543 , -0.21191406, -0.13964844, ..., -0.0480957 ,
         0.13769531, -0.03198242],
       [ 0.11279297, -0.13085938,  0.06689453, ..., -0.17285156,
        -0.07910156,  0.05786133],
       [ 0.24609375,  0.19628906, -0.17871094, ...,  0.13964844,
        -0.0025177 ,  0.35742188]], dtype=float32)

In [13]:
distance_matrix = cdist(np_keywords, centroid_coordinate, 'euclidean') # calculate the distance of all keyword and centroids
print(len(distance_matrix)) # cityblock is the Manhattan distance https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cdist.html
print(distance_matrix.shape[1])
distance_matrix

112
16


array([[3.63590555, 3.46220146, 4.04119684, ..., 3.87171731, 3.21588726,
        3.73691739],
       [3.73552992, 3.13154992, 3.68121998, ..., 4.10377914, 3.03846004,
        3.70603255],
       [4.24413649, 3.90725089, 4.48389202, ..., 4.42399068, 3.9656758 ,
        4.35699846],
       ...,
       [3.89322683, 3.4103483 , 4.12336569, ..., 4.16977659, 3.32904114,
        4.1542562 ],
       [4.07600827, 3.57203145, 4.4215931 , ..., 4.65120239, 3.62667309,
        4.16018038],
       [3.4021537 , 3.03228619, 3.64408251, ..., 3.60919999, 2.92530612,
        3.63297197]])

In [14]:
labels = np.array([np.argmin(i) for i in distance_matrix]) # set firts labels to all keyword in function of the distance
labels

array([ 5, 14, 12, 14,  5,  5, 14, 14,  5,  3,  5, 15,  5,  5, 14, 14,  1,
       14,  4, 15,  5,  8, 14,  5, 14,  5, 11,  5, 14,  4,  5,  5, 14,  5,
        5,  8,  5, 14,  9,  1, 14,  5, 13,  5,  1,  1, 14, 14, 14, 14, 14,
        5, 14, 15,  5,  1,  2, 14,  7,  8, 14, 14, 14,  1,  5, 10, 14, 14,
        5,  4, 14,  5,  5,  5,  5,  8,  1, 14,  5,  5, 14,  1,  3,  5,  5,
        5, 14, 14, 14,  5,  8,  0, 11,  8,  5,  5, 14, 14, 14,  6, 13, 10,
        5,  5, 14, 14, 14,  5,  3,  5,  5,  8])

In [15]:
max_iter=500 # set max number of iteration 
for it in range(max_iter):
    print(f"{it}/{max_iter}")
    centroid_coordinate = []
    for idx in range(n_cluster):
        # Updating Centroids by taking mean of Cluster it belongs to
        temp_cent = np_keywords[labels == idx].mean(axis=0) # axis=0 all vector 300x1
        centroid_coordinate.append(temp_cent)

    centroid_coordinate = np.vstack(centroid_coordinate)  # Updated Centroids
    
    distance_matrix = cdist(np_keywords, centroid_coordinate, 'euclidean') # recalculate the distance

    old_labels = labels.copy()
    labels = np.array([np.argmin(i) for i in distance_matrix])

    # Check for convergence
    if all(labels == old_labels):
        print('Reached convergence at iteration nr', it)
        break

0/500
1/500
2/500
Reached convergence at iteration nr 2


In [16]:
labels # display the clustering associated with each keyword (from 0 to 15 clusters)

array([ 5,  5, 12, 14,  5,  5, 14, 14,  5,  3,  5, 15,  5,  5, 14, 14,  1,
       14,  4, 15, 14,  8, 14,  5, 14,  5, 11,  5, 14,  4,  5,  5, 14,  5,
        5,  8,  5, 14,  9,  1, 14,  5, 13,  5,  1,  1, 14, 14, 14, 14, 14,
        5, 14, 15,  5,  1,  2, 14,  7,  8, 14, 14, 14,  1,  5, 10, 14, 14,
        5,  4, 14,  5,  5,  5,  5,  8,  1, 14,  5,  5, 14,  1,  3,  5,  5,
        5,  5, 14, 14,  5,  8,  0, 11,  8,  8,  5, 14, 14, 14,  6, 13, 10,
        8,  5, 14, 14, 14,  5,  3,  5,  1,  8])

In [17]:
# save inside a dictionary the set of keyword correlated to each cluster
cluster_result={0:{},1:{},2:{},3:{},4:{},5:{},6:{},7:{},8:{},9:{},10:{},11:{},12:{},13:{},14:{},15:{},16:{},17:{},18:{},19:{}}
for i in range(len(labels)):
    cluster_result[labels[i]][len(cluster_result[labels[i]])] = set_of_keywords[i]   

cluster_result

{0: {0: 'status'},
 1: {0: 'color',
  1: 'humidity',
  2: 'light',
  3: 'lighting',
  4: 'mood',
  5: 'panel',
  6: 'sensor',
  7: 'shed',
  8: 'weather'},
 2: {0: 'motion'},
 3: {0: 'bed', 1: 'sleep', 2: 'waking'},
 4: {0: 'control', 1: 'energy', 2: 'power'},
 5: {0: 'account',
  1: 'air',
  2: 'anything',
  3: 'anytime',
  4: 'arrival',
  5: 'burst',
  6: 'case',
  7: 'change',
  8: 'departure',
  9: 'door',
  10: 'end',
  11: 'event',
  12: 'everything',
  13: 'graph',
  14: 'home',
  15: 'house',
  16: 'jawbone',
  17: 'life',
  18: 'middle',
  19: 'money',
  20: 'panic',
  21: 'place',
  22: 'push',
  23: 'region',
  24: 'report',
  25: 'row',
  26: 'set',
  27: 'setting',
  28: 'someone',
  29: 'something',
  30: 'song',
  31: 'sound',
  32: 'state',
  33: 'switch',
  34: 'turn',
  35: 'valve',
  36: 'warning'},
 6: {0: 'thermostat'},
 7: {0: 'multi'},
 8: {0: 'day',
  1: 'hour',
  2: 'night',
  3: 'schedule',
  4: 'station',
  5: 'sunrise',
  6: 'sunset',
  7: 'tomorrow',
  8: '

In [18]:
myFile = open("analysis_NEW/centroids.txt","w") # save it
for i in range(len(centroid_coordinate)):
               myFile.write(str(centroid_coordinate[i]))
               myFile.write("\n")

In [19]:
for i in range(n_cluster):
    print(f"keywords appartenenti al cluster {i}: {len(cluster_result[i])}\n")

keywords appartenenti al cluster 0: 1

keywords appartenenti al cluster 1: 9

keywords appartenenti al cluster 2: 1

keywords appartenenti al cluster 3: 3

keywords appartenenti al cluster 4: 3

keywords appartenenti al cluster 5: 37

keywords appartenenti al cluster 6: 1

keywords appartenenti al cluster 7: 1

keywords appartenenti al cluster 8: 9

keywords appartenenti al cluster 9: 1

keywords appartenenti al cluster 10: 2

keywords appartenenti al cluster 11: 2

keywords appartenenti al cluster 12: 1

keywords appartenenti al cluster 13: 2

keywords appartenenti al cluster 14: 36

keywords appartenenti al cluster 15: 3



In [20]:
myFile = open("analysis_NEW/cluster_result.txt","w") # save
for i in range(n_cluster):
    myFile.write(f"\ncluster {i}:(tot keywords found:{len(cluster_result[i])})\n")
    myFile.write(f"keyword of the cluster:")
    for j in range(len(cluster_result[i])):
        myFile.write(f" {cluster_result[i][j]}")
    
    # the centroid coordinates it's not a real word because is a mean between other words
    myFile.write(f"\ncentroid fund:{model.most_similar(centroid_coordinate[i])}\n")
    myFile.write(f"")

    
myFile.close()

In [ ]:
#trovati cluster ottimi!! - analisi carta e penna dei risulatati
physical_channel_centroids = {}

In [ ]:
physical_channel_centroids["Temperature"] = np.array(centroid_coordinate[13])
physical_channel_centroids["Humidity"] = np.array(centroid_coordinate[12])
physical_channel_centroids["Illumination"] = np.array(centroid_coordinate[3])
physical_channel_centroids["Location"] = np.array(centroid_coordinate[6])
physical_channel_centroids["Motion"] = np.array(centroid_coordinate[14])
physical_channel_centroids["Smoke"] = np.array(centroid_coordinate[15])
physical_channel_centroids["Leakage"] = np.array(centroid_coordinate[0])

In [ ]:
physical_channel_centroids["Temperature"]

In [ ]:
#save the cluster centroids inside a JSON file :)
myFile = open("analysis_NEW/physical_channel_2.JSON","w") # save
myFile.write("{\n")
for key in physical_channel_centroids.keys():
    myFile.write(f"\"{key}\":{repr(physical_channel_centroids[key])},\n")
    
myFile.write("}")
myFile.close()

In [ ]:
# test aggiuntivi altro algoritmo di clustering **

In [ ]:
from sklearn.cluster import KMeans
n_clusters=16
# Init estimator and fit it on the data
sk_kmeans = KMeans(n_clusters=16, max_iter=max_iter).fit(np_keywords)

# Get predicted labels and centroids
sk_labels = sk_kmeans.labels_
sk_centroids = sk_kmeans.cluster_centers_

In [ ]:
sk_labels

In [ ]:
sk_centroids

In [ ]:
# save inside a dictionary the set of keyword correlated to each cluster
sk_cluster_result={0:{},1:{},2:{},3:{},4:{},5:{},6:{},7:{},8:{},9:{},10:{},11:{},12:{},13:{},14:{},15:{},16:{},17:{},18:{},19:{},
                  20:{},21:{},22:{},23:{},24:{}}
for i in range(len(np_keywords)):
    sk_cluster_result[sk_labels[i]][len(sk_cluster_result[sk_labels[i]])]=key_list[i]   

cluster_result

In [ ]:

myFile = open("analysis_NEW/sk_cluster_result.txt","w") # save
for i in range(n_cluster):
    myFile.write(f"\ncentroid cluster {i}:(tot keywords found:{len(sk_cluster_result[i])})\n")
    myFile.write(f"keyword of the cluster:")
    for j in range(len(sk_cluster_result[i])):
        myFile.write(f" {sk_cluster_result[i][j]}")
    
    myFile.write(f"\ncentroid fund:{model.most_similar(sk_centroids[i])}\n")
    myFile.write(f"")

    
myFile.close()